In [51]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [52]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [53]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [54]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver'

In [55]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\Santi\AppData\Local\Temp/ipykernel_8496/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [56]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [57]:
# Getting HTML content for news listing page
soup = make_request(browser, '/archivo/politica/2')

In [58]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [59]:
# Getting blocks from layout
blocks = layout.find('section').find_all(recursive = False)
print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagintion

2


In [60]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
len(cards)

6

In [61]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [62]:
news

[{'title': 'Corte Constitucional, último intento para evitar cambios a la Ley de Garantías',
  'relative_path': '/politica/corte-constitucional-ultima-instancia-para-evitar-cambios-a-la-ley-de-garantias/'},
 {'title': '“No vamos a reconocerlo”: Duque cerró puertas a restaurar relaciones con Maduro',
  'relative_path': '/politica/no-vamos-a-reconocerlo-duque-cerro-puertas-a-restaurar-relaciones-con-maduro/'},
 {'title': 'Doblemente desaparecido: 35 años buscando al líder estudiantil José Mejía',
  'relative_path': '/politica/doblemente-desaparecido-35-anos-buscando-al-lider-estudiantil-jose-mejia/'},
 {'title': '“Sería injusto que se me tilde por el pasado del partido”: Caterine Ibargüen',
  'relative_path': '/politica/seria-injusto-que-se-me-tilde-por-el-pasado-del-partido-caterine-ibarguen/'},
 {'title': 'Senado aprobó proposición para restablecer relaciones con Venezuela ',
  'relative_path': '/politica/senado-aprobo-proposicion-para-restablecer-relaciones-con-venezuela/'},
 {'title'

In [63]:
for n in news:
    # Getting HTML content for news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [64]:
news

[{'title': 'Corte Constitucional, último intento para evitar cambios a la Ley de Garantías',
  'relative_path': '/politica/corte-constitucional-ultima-instancia-para-evitar-cambios-a-la-ley-de-garantias/',
  'datetime': '21 Oct 2021  - 8:13 a.\xa0m.',
  'author': 'Redacción Política',
  'summary': 'Desde la Misión de Observación Electoral hasta congresistas de diferentes corrientes políticas pidieron al alto tribunal revisar la modificación que fue aprobada esta semana por Senado y Cámara. Los cambios, según dicen, abren la puerta a que se usen recursos públicos para favorecer campañas políticas en las elecciones de 2022.',
  'full_text': 'La modificación a la Ley de Garantías, aprobada esta semana en el Congreso en medio de la discusión del presupuesto de 2022, generó bastante controversia entre diferentes sectores. Como advirtieron muchas voces antes de que se realizara el debate, avalar las modificaciones iba a tener un mayor impacto en el tema electoral que en el objetivo de ese ca

In [65]:
# Storing extracted information for further analysis
collection.insert_many(news)